In [1]:
import requests, bs4
import re
from bs4 import BeautifulSoup as bs
from time import sleep

import pandas as pd
import numpy as np

In [2]:
# create lists for each feature
series_list = []
brand_list = []
panel_type_list = []
efficiency_list = []
power_list = []
weight_list = [] 
region_list = [] 
dimension_list = [] 
price_list = []

In [3]:
# adjust page numbers 
pages = [i for i in range(1,560)]

In [5]:
url_base = 'https://www.enfsolar.com/pv/panel/'

url_list = [url_base+str(i) for i in pages]

for url in url_list:
    sleep(1)
    page = requests.get(url).text
    soup = bs(page)

    for item in soup.find_all(class_ = 'enf-product-name'):
        series = item.get('title')
        series_list.append(series)
    
    for item in soup.find_all(class_ = 'enf-product-name'):
        brand = item.next.next.next.text.strip()
        brand_list.append(brand)
        
    for item in soup.find_all(class_ = 'enf-pd-list-table'):
        panel_type = item.find(class_ = 'enf-yellow').text.strip()
        panel_type_list.append(panel_type)
        
    for item in soup.find_all('td', text = re.compile('Panel Efficiency')):
        efficiency = item.next.next.next.next.strip()
        efficiency_list.append(efficiency)
        
    for item in soup.find_all(text = re.compile('Power Range')):
        power_range = item.next.next.next.text.strip()
        power_list.append(power_range)
        
    for weight in soup.find_all(text = re.compile('Weight:')):
        weight =  weight.next.next.text.strip()
        weight_list.append(weight)
        
    for item in soup.find_all(text = re.compile("Region")):
        region = item.next.next.text.strip()
        region_list.append(region)
        
    for item in soup.find_all(text = re.compile("Panel Dimension")):
        dimension = item.next.next.text.strip()
        dimension_list.append(dimension)
        
    for item in soup.find_all(text = re.compile("From")):
        price = item.next.text.strip()
        price_list.append(price)

        

In [6]:
#check if all lists have the same length
#price info is often missing

for list in [series_list, brand_list, panel_type_list, efficiency_list,
power_list, weight_list, region_list, dimension_list, price_list]:
    print(len(list))

11163
11163
11163
11163
11163
11163
11163
11163
2055


In [7]:
#combine into single df
solar_dict = []

for i in range(len(price_list)):
    solar_dict_i = {'series': series_list[i],
                    'brand': brand_list[i],
                    'panel_type': panel_type_list[i],
                    'efficiency': efficiency_list[i],
                    'power': power_list[i],
                    'weight': weight_list[i],
                    'region': region_list[i],
                    'dimension': dimension_list[i],
                    'price': price_list[i]
                   }
    solar_dict.append(solar_dict_i)

In [8]:
df_jan17 = pd.DataFrame(solar_dict)

### Save dataframe

In [9]:
df_jan17.to_csv('df_jan17.csv')

In [10]:
# note: there are only 2055 datapoints on price!!!

df_jan17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055 entries, 0 to 2054
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   series      2055 non-null   object
 1   brand       2055 non-null   object
 2   panel_type  2055 non-null   object
 3   efficiency  2055 non-null   object
 4   power       2055 non-null   object
 5   weight      2055 non-null   object
 6   region      2055 non-null   object
 7   dimension   2055 non-null   object
 8   price       2055 non-null   object
dtypes: object(9)
memory usage: 144.6+ KB


In [13]:
df_jan17.isna().sum()

series        0
brand         0
panel_type    0
efficiency    0
power         0
weight        0
region        0
dimension     0
price         0
dtype: int64